In [27]:
from neural_network import *

from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

In [28]:
input_size1 = 5
output_size1 = 2
input_size2 = 3
output_size2 = 2

filename = 'model.pt'

NN = Neural_network(input_size1,output_size1,input_size2,output_size2)

NN.load(filename)
NN.eval()

Neural_network(
  (net1): Sequential(
    (0): Linear(in_features=5, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.0, inplace=False)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.0, inplace=False)
    (11): Linear(in_features=50, out_features=2, bias=True)
  )
  (net2): Sequential(
    (0): Linear(in_features=10, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.0, inplace=False)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.0, inplace=False)
    (11): Linear(in_features=50, 

In [29]:
file_path = "../Data/01_Inicial_v8.xlsx"

D = Data_loader(file_path,seed=0)

In [30]:
inputs = np.zeros((0,input_size1+input_size2))
targets = np.zeros((0,output_size1+output_size2))
preds = np.zeros((0,output_size1+output_size2))

for X, Y_target in D.get_batch2(batch_size=10):
    Y_pred = NN(X)

    X1 = X[:,:input_size1].cpu().numpy()
    X2 = X[:,input_size1:].cpu().numpy()

    Y1_target = Y_target[:,:output_size1].cpu().numpy()
    Y2_target = Y_target[:,output_size1:].cpu().numpy()

    Y1_pred = Y_pred[:,:output_size1].cpu().detach().numpy()
    Y2_pred = Y_pred[:,output_size1:].cpu().detach().numpy()

    X1,X2,Y1_target,Y2_target = D.denormalize_values(X1=X1,Y1=Y1_target,X2=X2,Y2=Y2_target)
    _,_,Y1_pred,Y2_pred = D.denormalize_values(Y1=Y1_pred,Y2=Y2_pred)

    inputs = np.vstack((inputs,np.hstack((X1,X2))))

    targets = np.vstack((targets,np.hstack((Y1_target,Y2_target))))
    preds = np.vstack((preds,np.hstack((Y1_pred,Y2_pred))))

n_train = inputs.shape[0]

for X, Y_target in D.get_batch2(batch_size=10,validation=True):
    Y_pred = NN(X)

    X1 = X[:,:input_size1].cpu().numpy()
    X2 = X[:,input_size1:].cpu().numpy()

    Y1_target = Y_target[:,:output_size1].cpu().numpy()
    Y2_target = Y_target[:,output_size1:].cpu().numpy()

    Y1_pred = Y_pred[:,:output_size1].cpu().detach().numpy()
    Y2_pred = Y_pred[:,output_size1:].cpu().detach().numpy()

    X1,X2,Y1_target,Y2_target = D.denormalize_values(X1=X1,Y1=Y1_target,X2=X2,Y2=Y2_target)
    _,_,Y1_pred,Y2_pred = D.denormalize_values(Y1=Y1_pred,Y2=Y2_pred)

    inputs = np.vstack((inputs,np.hstack((X1,X2))))

    targets = np.vstack((targets,np.hstack((Y1_target,Y2_target))))
    preds = np.vstack((preds,np.hstack((Y1_pred,Y2_pred))))

train = np.zeros((inputs.shape[0],1))
train[:n_train,0] = 1



In [31]:

input_names = D.input_columns_DDO + D.input_columns_RBDO
output_names = D.output_columns_DDO + D.output_columns_RBDO
output_names_target = [f'{c}_target' for c in output_names]
output_names_pred = [f'{c}_pred' for c in output_names]

df = pd.DataFrame(np.hstack((inputs,targets,preds,train)), columns=input_names+output_names_target+output_names_pred+['train'])

df.to_csv('output.csv', sep=',', index=False)

In [32]:
df

,c,phi,gamma,HV,H,c_cov,phi_cov,gamma_cov,V1_target,FS_target,V2_target,beta_target,V1_pred,FS_pred,V2_pred,beta_pred,train
0,8.0,28.0,20.0,1.5,20.0,15.000001,26.0,9.0,149.322288,1.276,155.007471,0.921500,147.427593,1.240782,168.087171,0.869577,1.0
1,10.0,20.0,16.0,2.0,10.0,49.999999,30.0,11.0,142.508555,1.414,116.303234,1.068700,146.352585,1.376125,118.771390,1.037280,1.0
2,17.0,27.0,16.0,1.5,30.0,28.000000,19.0,5.0,417.841359,1.431,417.841360,1.927200,413.661137,1.386121,422.681111,1.674591,1.0
3,22.0,23.0,16.0,1.0,30.0,18.000000,16.0,8.0,370.395290,1.091,370.395291,0.582400,364.988394,1.106750,335.850056,0.693699,1.0
4,11.0,21.0,16.0,2.5,20.0,24.000000,15.0,11.0,783.695040,1.439,783.695027,2.541128,836.378953,1.421334,789.601264,2.479869,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,20.0,30.0,18.0,1.5,30.0,34.000000,23.0,10.0,417.841359,1.588,417.841360,1.984267,368.146026,1.651385,434.550008,2.214721,0.0
265,10.0,20.0,15.0,2.5,30.0,23.000000,26.0,3.0,1524.869891,1.260,1623.708052,0.880788,1542.000004,1.281206,1487.665330,1.093541,0.0
266,28.0,25.0,18.0,1.5,20.0,46.000000,21.0,4.0,276.248607,1.703,164.598103,1.746300,231.500257,1.690457,106.385711,1.763666,0.0
267,21.0,20.0,19.0,1.5,20.0,39.000000,18.0,10.0,276.248607,1.285,217.825126,1.125700,328.888001,1.240616,205.059353,1.072175,0.0
